# Data-Driven Research Assignment 2: Topic Modeling
This notebook contains the second, collaborative, graded assignment of the 2023 Data-Driven Research course. In this assignment you'll use a topic modeling tool in order to uncover the ''topics'' of a large set of reviews of popular films. 

To complete the assignment, complete **Part 1, Part 2, Part 3 and Part 4** of the **Your Model** section at the end.

This is a collaborative assignment. In the text cell below, please include all the names of your group members.

If you used code or a solution from the internet (such as StackOverflow) or another external resource, please make reference to it (in any format). Unattributed copied code will be considered plagiarism and therefore fraud.


**Authors of this answer:**

# 1. Introduction

You'll use a Topic Modelling tool from Gensim, a popular library for topic modelling in Python, though these days mainly known for its implementation of Word2Vec to train word embeddings (dense representations). Using this library, you will model topics based on reviews of popular films. The reviews are stored in plain text files, organized by film and rating. The aim of this exercise is to familiarize you with the topic modeling process and its output and to get insight in what kinds of topics are modeled.

# 2. Preparation

This assignment comes with the following files:


1.   The reviews of the films. This is the data in which we want to find topics. They are found in the movie2k/txt_sentoken directory. There are then two types: negative reviews (neg directory) and positive reviews (pos directory). The reviews are already tokenized.
2.   Stopword list files. They are found in the stopwords directory.

Let's start by loading the movie reviews from the files (I'll do it for you):

In [ ]:
import os

def load_reviews(folder_path):
    reviews = [] #Make a list to put the reviews in
    reviewnames = [] # Make a list to put the review filenames in (to be able to look them up later)
    tokens = 0 #Make a counter for the number of tokens
    
    for file in os.listdir(folder_path):
        #Loop through all the text files in the folder, each containing one review
        
        if not file.endswith('.txt'):  #Only read text files
            continue

        file_path = os.path.join(folder_path, file)

        #Open the text file and read its contents
        with open(file_path, encoding='utf-8') as infile:
            review = infile.read()
        reviewnames.append(file)
            
        # Turn the string with the review into a list of words (this is easy because it is already tokenized)
        review = review.split()
        # And add it to the list
        reviews.append(review)
        # To count the number of tokens processed so far
        tokens = tokens + len(review)

    print(f"Loaded reviews from {folder_path} containing {tokens} tokens in total.") 
    return reviews, reviewnames
        
folder_path = "movie2k/txt_sentoken"
    
movie_reviews_pos, movie_reviewnames_pos = load_reviews(folder_path + "/pos") #Load the positive reviews
movie_reviews_neg, movie_reviewnames_neg = load_reviews(folder_path + "/neg") #Load the negative reviews

movie_reviews = movie_reviews_pos + movie_reviews_neg #Combine the lists of positive and negative reviews into one
movie_reviewnames = movie_reviewnames_pos + movie_reviewnames_neg #The same for the list of filenames

If you are working on Google Colab, you will probably have to change the path to the files to something that Google Colab has access to. For example, you could put the files on your Google Drive and then load them from there, as we did in Coding the Humanities. For more details about how to work with files in Python and load them from Google Drive, have a look at the Coding the Humanities course notebook on Files: https://github.com/bloemj/2023-coding-the-humanities/blob/main/notebooks/4_ReadingAndWritingFiles.ipynb

How to load files off Google Drive is explained at the beginning there.

## Preprocessing

Now that we have loaded the text, you might want to perform some pre-processing steps to be able to create a better bag-of-words model in which all forms of a word are mapped to a single number. For example, you could remove the punctuation characters, or you could perform lemmatization or stemming, which we discussed in the lecture. This would be the place to do it by writing a preprocessing function that accepts a list of movie reviews as its argument and returns a preprocessed list of movie reviews. Feel free to use your knowledge of text normalization from Coding the Humanities or the functions you wrote then. Here is some information on how to perform stemming with NLTK: https://www.nltk.org/howto/stem.html

You can also try other forms of preprocessing, if you are able to do it.

Make sure to also keep the unmodified reviews, so you can compare the results with preprocessing and without preprocessing.

**Part 1: Preprocessing**

You can also skip this part for now - it is not required to perform the topic modelling, but you will get better results.

In [ ]:
from nltk.stem import *
import string

preprocessed_movie_reviews = []

# your code here

# 3. Topic Modelling using Gensim

Gensim offers an implementation of Latent Dirichlet Allocation (LDA), the most popular topic modelling algorithm, which we discussed in the lecture. If you are working on Google Colab, it is normally already installed there. Otherwise, you can install it with `pip install --upgrade gensim` or if you are using Conda, `conda install -c conda-forge gensim`.

Let's load it, and some other things we use:

In [ ]:
import gensim
import gensim.corpora as corpora
import gensim.models as models
import itertools
from operator import itemgetter
print(gensim.__version__)

## Constructing the bag-of-words model

The `gensim.corpora.Dictionary()` class allows you to map words to numbers, which is what we need to make a bag-of-words model. In particular, the doc2bow() function converts a collection of words to a bag-of-words representation:

In [ ]:
movie_dictionary = corpora.Dictionary(movie_reviews)
movie_bow_corpus = [movie_dictionary.doc2bow(d) for d in movie_reviews]

Let's see what happened:

In [ ]:
print('Number of unique tokens in the dataset:', len(movie_dictionary))

#Checking the first 11 words in the bag-of-words model
print('\nThe first 11 words in the bag-of-words model:')
print(dict(itertools.islice(movie_dictionary.token2id.items(), 12)))

#Checking the first 100 words of the first review
print('\nThe start of the first review:')
print(movie_reviews[0][:100])
#And the filename of that review is...
print('\nThe filename of the first review:')
print(movie_reviewnames[0])

#Which words are used in that review?
print('\nMost frequent words in the first review:')
for i, freq in sorted(movie_bow_corpus[0], key=itemgetter(1), reverse=True)[:20]:
    print(movie_dictionary[i], "-->", freq)
print("...")

## The topic model

Now, we can train our LDA model on this bag-of-words data by using `gensim.models.ldamodel.LDAModel()`.

This model can take various parameters that specify what kind of model gets made. Some important ones:


* num_topics: how many topics do we want? In what follows, we set the number of topics to 5, because we want to have a few topics that we can interpret, but the number of topics is data and application-dependent;
* id2word: our bag-of-words dictionary needed to map ids to strings;
* passes: how often we iterate over the entire corpus (default = 1). In general, the more passes, the higher the accuracy. This number is also called epochs in Artificial Intelligence and Machine Learning.

Let's first make a model that finds 5 topics, and tries 25 times to improve its estimate. This code may take a while to run, as it is the process that creates the topic model. If it takes too long, you can reduce the number of passes, but the topics might be worse.

In [ ]:
reviews_ldamodel = models.ldamodel.LdaModel(movie_bow_corpus, num_topics=5, id2word = movie_dictionary, passes=25)

And let's have a look! An easy way to inspect the created topics is by using the `show_topics()` method, which prints the most representative word for each topic along with their probability.

In [ ]:
reviews_ldamodel.show_topics(num_words=8) #Show the top 8 words for each topic

There we go, we have a topic model. However, you can probably see that it is far from perfect and some uninteresting 'words' appear there. Now, it is your turn to make it better!

## Your model

**Part 1: Preprocessing**

Show the effect of your preprocessing by also making a topic model for your preprocessed_movie_reviews. First, you make a bag-of-words model and then the LdaModel, as above. Feel free to go back to your preprocessing code above and update it based on what you saw from the show_topics function applied to the initial model.

Try to make a model with 8 topics, and show the top 8 words for each topic. **Assign the model to a new variable with a sensible name** (avoid overwriting the previous models).

Also for the dictionary and corpus, **give the variables different and expressive names to avoid overwriting the other ones**. Otherwise, you will get confused between your different topic models.

In [ ]:
# your code here

**Part 2: Stopwords**

The topics you saw so far are probably mostly made up of stopwords such as "the". As discussed in the lecture, our results will probably be more interesting if we get rid of them.

We have included 3 generic lists of stopwords: the default list of the tool Mallet, a shorter frequent word list used in search applications (Snowball stemmer), and the top 10,000 words based on Google n-grams (in frequency order, select as many lines as you want). Gensim and NLTK also have stopword lists.

Make a function that accepts the path to a stopwords file (e.g. `stopwords/standard-mallet-en.txt`), and returns a list of stopwords.

In [ ]:
def load_stopwords(filename):
    
    #your code here
    
    return stopword_list

stopword_list = load_stopwords("stopwords/standard-mallet-en.txt")

Then, make a function that takes a stopword list and a list of reviews (e.g. `preprocessed_movie_reviews`). The function should remove all stopwords from all the reviews, returning a list of the reviews without stopwords. This code may be a bit slow if you have many stopwords, since there is a lot of data to process.

In [ ]:
def filter_stopwords(stopword_list, movie_reviews):
    
    filtered_movie_reviews = []
    
    for review in movie_reviews:
        
        #your code here
        
    return filtered_movie_reviews

filtered_movie_reviews = filter_stopwords(stopword_list, preprocessed_movie_reviews)

Lastly, let's make another topic model with this filtered data! Again, you make a bag-of-words model and then the LdaModel, as above.

Try to make a model with 8 topics, and show the top 8 words for each topic. Assign the model to a new variable with a sensible name (avoid overwriting the previous models).

In [ ]:
# your code here

**Part 3: Experimentation**

Are these general stopword lists sufficient? We are working in the movie review domain, meaning that we may have other uninformative stopwords than in the general domain, such as the word 'movie'. Some key experimentation is to add specific stopwords for the movie review domain, which would occur frequently in all (or most) of the clusters. Note that removing words will not just hide these words, but lead to (even very) different topics and different top ranked reviews.

**Make your own domain-specific stopwords file** by taking one of the existing ones and adding your own stopwords (make sure that the stopword file is saved as a plain text file). Think about what stopwords are in this domain (e.g., the word film is not a stopword in general, but it will occur in essentially every film review).

Re-use the functions you previously made to load your own stopwords file and filter the movie reviews. Then, make another topic model with your new filtering and show the top 8 words for each topic.

In [ ]:
my_stopword_list = load_stopwords("stopwords/my_movie_stopwords.txt")
domainfiltered_movie_reviews = filter_stopwords(my_stopword_list, preprocessed_movie_reviews)

# your code here

Now, you should have 3 models (or more): one without any stopword filtering, one with the standard stopword filtering and one with the domain-filtered stopwords using the list you modified yourself. Compare the topics found by the three models (just looking at them is fine, no need to code a comparison).

Do the topics look better with stopword filtering and with domain-specific stopword filtering? At this point, do the resulting topics correspond to particular film genres you have expected?

(your answer here)

Increase the number of topics. What happens with the topics if you model very few or very many topics? (answer in a text box). Assign the model(s) to a new variable with a sensible name (avoid overwriting the previous models).

In [ ]:
# your code here

(your answer here)

Increase the number of topic words printed to get more information per topic.  Is it easier to make sense of a topic if you look further down the list, or are the initial words more clear?

In [ ]:
# your code here

(your answer here)

If you are interested, you can also experiment with the difference between positive and negative reviews.

### Part 4: Evaluation

There are a few numbers we can compute that indicate the quality of a topic model, such as [perplexity and coherence](https://github.com/ccs-amsterdam/r-course-material/blob/master/tutorials/R_text_LDA_perplexity.md). For perplexity, a lower number means a better model, and for coherence, a higher number is better. Try computing these scores for your models, and see which is the best one according to the numbers

In a real project, you should compute these numbers over a separate part of the dataset (the test set) for a proper evaluation, but for simplicity and because we have not talked about this in the lecture we will skip that here.

In [ ]:
from gensim.models import CoherenceModel

#Compute perplexity for the basic model on the bag-of-words representation of the reviews:
print('Perplexity: ', reviews_ldamodel.log_perplexity(movie_bow_corpus))  

# Compute coherence score on the same:
coherence_model_lda = CoherenceModel(model=reviews_ldamodel, texts=movie_reviews, dictionary=movie_dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence score: ', coherence_lda)

In [ ]:
# Compute these numbers for your other models here

However, just comparing numbers is not very interpretable. We will choose our topic model with the highest coherence score and validate the evaluation.

Using the top 20 topic words for each topic in the model with the highest coherence score, pick at least 5 topic numbers and determine what film genres (in an informal sense) they represent, i.e. think of a meaningful label for the topic. Write down the topic number and your topic label. Is it easy to guess what the topic represents? For how many topics are you fairly confident, for how many do you have to make a guess, and for how many do you have no real clue.

(your answer here)

In [ ]:
reviews_ldamodel.get_term_topics("the", minimum_probability = 1e-3)

Do this for your own best model and the labels you just picked. For each of your topic labels, if the probability for the label is the highest for the topic number you wrote down, your guess was probably correct. Did you guess a suitable label for every topic?

In [ ]:
# your code here

(your answer here)

In a real project, you would also want to validate your topics by examining the reviews that are most strongly associated with that topic. You can see what documents have what topics using the get_document_topics() method. Here we look at the topics for the first document in the model (change the name of the model to yours):

In [ ]:
reviews_ldamodel.get_document_topics(movie_bow_corpus[0], minimum_probability = 0)

Or for the first 20 of them:

In [ ]:
for i, doc_topics in enumerate(reviews_ldamodel.get_document_topics(movie_bow_corpus)):
    if i >= 20:
        break
    print(f"Topics for the review {movie_reviewnames[i]}: {doc_topics}")

But this assignment is already long enough so I will not ask you to report on this too!